# Chunking Strategies I: Text based chunking

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv
import tiktoken
from IPython.display import display, Markdown

from src import utils

# Params

In [3]:
TOKENIZER_NAME ="cl100k_base"

In [4]:
path_data = Path() / "data"
path_input = path_data / "interim"
path_output = path_data / "processed" / "chunks"  # Store chunks by experiment

path_output.mkdir(exist_ok=True, parents=True)

# Environment Variables

In [5]:
load_dotenv()

True

## Intro Langchain 

[Langchain Ecosystem](https://python.langchain.com/svg/langchain_stack_112024.svg)


A text splitter will follow some rules to split each document. In addition, is a convenient method to pass a list documents and get a list of chunks in a simple way

[Text Splitters](https://python.langchain.com/docs/concepts/text_splitters/)

In [6]:
from langchain_core.documents import Document

document = Document(
    page_content="Hello, world! \n\n Let's Build a RAG",
    metadata={"source": "https://example.com"}
)

document

Document(metadata={'source': 'https://example.com'}, page_content="Hello, world! \n\n Let's Build a RAG")

In [7]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    chunk_size=10,
    chunk_overlap=0
)
texts = text_splitter.split_documents([document])

texts

Created a chunk of size 14, which is longer than the specified 10


[Document(metadata={'source': 'https://example.com'}, page_content='Hello, world!'),
 Document(metadata={'source': 'https://example.com'}, page_content="Let's Build a RAG")]

In [8]:
text_splitter.split_text("Hello, world!\n\nLet's Build a RAG")

Created a chunk of size 13, which is longer than the specified 10


['Hello, world!', "Let's Build a RAG"]

# Chunk by pages

In [9]:

import json

with open(path_output / "pages" / 'Althera.json', 'r') as f:
    dc_chunks = json.load(f)

In [10]:
corpus = [Document(**page) for page in dc_chunks]
Markdown(corpus[0].page_content)

# Un nuevo y fascinante vecino: Althéra 

## Índice

1. Historia del descubrimiento
2. Conoce a Althéra
3. Los soles de Althéra
4. Estructura general de Althéra
5. Planetas interiores
6. Planetas exteriores
7. Lunas y satélites menores
8. Fenómenos destacados
9. Habitabilidad y astrobiología
10. Conclusiones y perspectivas futuras

## 1. Historia del descubrimiento

### 1.1 Primeras observaciones y sospechas iniciales

El sistema binario Althéra ( $H D 4579 A B$ ) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb, dirigida por la astrofísica chilena Dra. Mariela Estay. La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas en órbita mutua y varios cuerpos orbitando de forma circumbinaria.

### 1.2 Confirmación mediante técnicas combinadas

En los meses siguientes, un consorcio internacional liderado por la Agencia Espacial Europea (ESA) y el Instituto Max Planck de Astronomía desplegó observaciones complementarias utilizando:

- ELT (Extremely Large Telescope) en el Desierto de Atacama para espectroscopía de alta resolución.
- Telescopio Espacial Nancy Grace Roman para fotometría de gran precisión en tránsitos.
- Interferometría de radio desde la red Very Long Baseline Array (VLBA) para afinar la distancia y parámetros orbitales del sistema.

In [11]:
len(corpus)

17

# Chunk with RCS

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter_rcs = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    encoding_name=TOKENIZER_NAME,
    chunk_size=256,
    chunk_overlap=25
)
corpus_rcs = text_splitter_rcs.split_documents(corpus)


In [13]:
Markdown(corpus_rcs[0].page_content)

# Un nuevo y fascinante vecino: Althéra 

## Índice

1. Historia del descubrimiento
2. Conoce a Althéra
3. Los soles de Althéra
4. Estructura general de Althéra
5. Planetas interiores
6. Planetas exteriores
7. Lunas y satélites menores
8. Fenómenos destacados
9. Habitabilidad y astrobiología
10. Conclusiones y perspectivas futuras

## 1. Historia del descubrimiento

### 1.1 Primeras observaciones y sospechas iniciales

In [14]:
Markdown(corpus_rcs[1].page_content)

### 1.1 Primeras observaciones y sospechas iniciales

El sistema binario Althéra ( $H D 4579 A B$ ) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb, dirigida por la astrofísica chilena Dra. Mariela Estay. La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas en órbita mutua y varios cuerpos orbitando de forma circumbinaria.

### 1.2 Confirmación mediante técnicas combinadas

En los meses siguientes, un consorcio internacional liderado por la Agencia Espacial Europea (ESA) y el Instituto Max Planck de Astronomía desplegó observaciones complementarias utilizando:

In [15]:
encoding = tiktoken.get_encoding(TOKENIZER_NAME)

stats_rcs = utils.corpus_stats(encoding, corpus_rcs)
stats_rcs.describe()

count     52.000000
mean     177.692308
std       63.117549
min       28.000000
25%      141.250000
50%      193.500000
75%      226.000000
max      255.000000
dtype: float64

In [16]:
# Join the corpus in a single doc to properly split

corpus_doc = Document(
    page_content="\n\n".join([doc.page_content for doc in corpus]),
    metadata={"source": "Althera"}
)

corpus_rcs_singleinput = text_splitter_rcs.split_documents([corpus_doc])
stats_rcs_singleinput = utils.corpus_stats(encoding, corpus_rcs_singleinput)
stats_rcs_singleinput.describe()

count     48.000000
mean     194.541667
std       51.092306
min       46.000000
25%      160.750000
50%      213.000000
75%      230.250000
max      255.000000
dtype: float64

In [17]:
stats_rcs.describe()

count     52.000000
mean     177.692308
std       63.117549
min       28.000000
25%      141.250000
50%      193.500000
75%      226.000000
max      255.000000
dtype: float64

## Markdown splitter

In [18]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

text_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on,
    strip_headers =False
    )
corpus_mds = text_splitter.split_text(corpus[0].page_content)

In [19]:
for i, doc in enumerate(corpus_mds):
    print(f"chunk: {i} ----------------")
    display(Markdown(doc.page_content))

chunk: 0 ----------------


# Un nuevo y fascinante vecino: Althéra  
## Índice  
1. Historia del descubrimiento
2. Conoce a Althéra
3. Los soles de Althéra
4. Estructura general de Althéra
5. Planetas interiores
6. Planetas exteriores
7. Lunas y satélites menores
8. Fenómenos destacados
9. Habitabilidad y astrobiología
10. Conclusiones y perspectivas futuras

chunk: 1 ----------------


## 1. Historia del descubrimiento  
### 1.1 Primeras observaciones y sospechas iniciales  
El sistema binario Althéra ( $H D 4579 A B$ ) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb, dirigida por la astrofísica chilena Dra. Mariela Estay. La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas en órbita mutua y varios cuerpos orbitando de forma circumbinaria.

chunk: 2 ----------------


### 1.2 Confirmación mediante técnicas combinadas  
En los meses siguientes, un consorcio internacional liderado por la Agencia Espacial Europea (ESA) y el Instituto Max Planck de Astronomía desplegó observaciones complementarias utilizando:  
- ELT (Extremely Large Telescope) en el Desierto de Atacama para espectroscopía de alta resolución.
- Telescopio Espacial Nancy Grace Roman para fotometría de gran precisión en tránsitos.
- Interferometría de radio desde la red Very Long Baseline Array (VLBA) para afinar la distancia y parámetros orbitales del sistema.

In [20]:
Markdown(corpus_mds[1].page_content)

## 1. Historia del descubrimiento  
### 1.1 Primeras observaciones y sospechas iniciales  
El sistema binario Althéra ( $H D 4579 A B$ ) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb, dirigida por la astrofísica chilena Dra. Mariela Estay. La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas en órbita mutua y varios cuerpos orbitando de forma circumbinaria.

In [21]:
headers_to_split_on = [
    ("#", "Header 1"),  # Big chunks
]

text_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on,
    strip_headers =False
    )
corpus_mds_big = text_splitter.split_text(corpus_doc.page_content)

for i, doc in enumerate(corpus_mds_big[:2]):
    print(f"chunk: {i} ----------------")
    display(Markdown(doc.page_content))

chunk: 0 ----------------


# Un nuevo y fascinante vecino: Althéra  
## Índice  
1. Historia del descubrimiento
2. Conoce a Althéra
3. Los soles de Althéra
4. Estructura general de Althéra
5. Planetas interiores
6. Planetas exteriores
7. Lunas y satélites menores
8. Fenómenos destacados
9. Habitabilidad y astrobiología
10. Conclusiones y perspectivas futuras  
## 1. Historia del descubrimiento  
### 1.1 Primeras observaciones y sospechas iniciales  
El sistema binario Althéra ( $H D 4579 A B$ ) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb, dirigida por la astrofísica chilena Dra. Mariela Estay. La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas en órbita mutua y varios cuerpos orbitando de forma circumbinaria.  
### 1.2 Confirmación mediante técnicas combinadas  
En los meses siguientes, un consorcio internacional liderado por la Agencia Espacial Europea (ESA) y el Instituto Max Planck de Astronomía desplegó observaciones complementarias utilizando:  
- ELT (Extremely Large Telescope) en el Desierto de Atacama para espectroscopía de alta resolución.
- Telescopio Espacial Nancy Grace Roman para fotometría de gran precisión en tránsitos.
- Interferometría de radio desde la red Very Long Baseline Array (VLBA) para afinar la distancia y parámetros orbitales del sistema.  
Fue el equipo del astrónomo estadounidense Dr. Jonathan Kepler-Saunders quien confirmó, mediante el método de velocidad radial ultraestable, la existencia de cinco planetas principales y varios cinturones de escombros.

chunk: 1 ----------------


# 1.3 Descubrimiento revolucionario de la zona habitable circumbinaria  
El hallazgo más impactante llegó en 2034, cuando la misión LUVOIR-B (Large UV/Optical/IR Surveyor) detectó la firma espectral de vapor de agua, oxígeno molecular y metano en la atmósfera de Aurelia III, un planeta ubicado en la zona habitable del sistema, orbitando a ambos soles. Este fue el primer caso documentado de un mundo potencialmente habitable en un sistema binario cercano - a tan solo 42,7 años luz de la Tierra-, lo que lo convierte en un candidato ideal para futuras misiones de exploración interestelar.  
### 1.4 Importancia científica y proyección futura  
El descubrimiento de Althéra revolucionó la astrobiología y la física orbital por tres razones clave:  
1. Dinámica circumbinaria estable - demostró que los planetas pueden mantener órbitas estables y climas equilibrados alrededor de dos soles, desafiando modelos anteriores.
2. Química atmosférica compleja - Aurelia III presenta una mezcla de gases que, en equilibrio fotoquímico, sugieren procesos biológicos o geoquímicos activos.
3. Proximidad relativa - su cercanía permite observaciones directas en la próxima década con telescopios como el Habitable Worlds Observatory (HWO) y misiones de espectrometría directa de superficie como Starshot Spectra.  
En 2036, la Unión Astronómica Internacional otorgó a este hallazgo el Premio Messier de Descubrimiento Astronómico y estableció el Programa Althéra, un plan coordinado de investigación que combina observaciones remotas, simulaciones climáticas y diseño de futuras sondas interestelares.  
Perfecto, aquí tienes la Sección 2 desarrollada con un tono divulgativo-científico y manteniendo coherencia con la historia previa:  
## 2. Conoce a Althéra  
### 2.1 Localización en la galaxia  
El sistema binario Althéra (HD 4579 AB ) se encuentra en el brazo de Orión de la Vía Láctea, a una distancia de 42,7 años luz de la Tierra. Su posición aparente en el cielo

In [22]:
stats_mds_big = utils.corpus_stats(encoding, corpus_mds_big)
stats_mds_big.describe()

count      16.000000
mean      566.437500
std       233.513445
min       212.000000
25%       441.500000
50%       530.500000
75%       616.250000
max      1153.000000
dtype: float64

# Docling Langchain Integration

Docling has a custom Chunker that works on top of Docling Document extraction

In [23]:
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType
from docling.chunking import HybridChunker

from docling_core.transforms.chunker.tokenizer.openai import OpenAITokenizer

path_input
FILE_PATH = [(path_data / "raw" / "Divulgacion Planetaria Althera.pdf").as_posix()]
EXPORT_TYPE = ExportType.DOC_CHUNKS

tokenizer = OpenAITokenizer(
    tokenizer=tiktoken.encoding_for_model("gpt-4o"),
    max_tokens=2048,  # context window length required for OpenAI tokenizers
)

loader = DoclingLoader(
    file_path=FILE_PATH,
    export_type=EXPORT_TYPE,
    chunker=HybridChunker(tokenizer=tokenizer),
)

corpus_docling = loader.load()

c:\Users\manua\Documents\repos\dslabs\dslab-docai\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-04 15:27:56,664 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-09-04 15:27:56,858 - INFO - Going to convert document batch...
2025-09-04 15:27:56,860 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 130b240947e51886114a732a44356305
2025-09-04 15:27:56,923 - INFO - Loading plugin 'docling_defaults'
2025-09-04 15:27:56,937 - INFO - Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-09-04 15:28:06,610 - INFO - Accelerator device: 'cpu'
2025-09-04 15:28:20,122 - INFO - Accelerator device: 'cpu'
2025-09-04 15:28:24,172 - INFO - Accelerator device: 'cpu'
2025-09-04 15:28:26,460 - INFO - Loading plugin 'docling_defaults'
2025-09-04 15:28:

In [24]:
stats_docling = utils.corpus_stats(encoding, corpus_docling)
stats_docling.describe()

count     52.000000
mean     170.865385
std       88.786377
min       26.000000
25%      105.250000
50%      154.000000
75%      224.500000
max      375.000000
dtype: float64

In [25]:
for i, doc in enumerate(corpus_mds_big[:3]):
    print(f"chunk: {i} ----------------")
    display(Markdown(doc.page_content))

chunk: 0 ----------------


# Un nuevo y fascinante vecino: Althéra  
## Índice  
1. Historia del descubrimiento
2. Conoce a Althéra
3. Los soles de Althéra
4. Estructura general de Althéra
5. Planetas interiores
6. Planetas exteriores
7. Lunas y satélites menores
8. Fenómenos destacados
9. Habitabilidad y astrobiología
10. Conclusiones y perspectivas futuras  
## 1. Historia del descubrimiento  
### 1.1 Primeras observaciones y sospechas iniciales  
El sistema binario Althéra ( $H D 4579 A B$ ) fue detectado por primera vez en el año 2032 durante una campaña de observación del Observatorio Espacial James Webb, dirigida por la astrofísica chilena Dra. Mariela Estay. La misión principal era estudiar la composición atmosférica de exoplanetas candidatos a la habitabilidad, pero un patrón anómalo en el flujo luminoso proveniente de la constelación de Orión llamó la atención del equipo. El análisis de curvas de luz reveló oscilaciones periódicas dobles, un indicio claro de la presencia de dos estrellas en órbita mutua y varios cuerpos orbitando de forma circumbinaria.  
### 1.2 Confirmación mediante técnicas combinadas  
En los meses siguientes, un consorcio internacional liderado por la Agencia Espacial Europea (ESA) y el Instituto Max Planck de Astronomía desplegó observaciones complementarias utilizando:  
- ELT (Extremely Large Telescope) en el Desierto de Atacama para espectroscopía de alta resolución.
- Telescopio Espacial Nancy Grace Roman para fotometría de gran precisión en tránsitos.
- Interferometría de radio desde la red Very Long Baseline Array (VLBA) para afinar la distancia y parámetros orbitales del sistema.  
Fue el equipo del astrónomo estadounidense Dr. Jonathan Kepler-Saunders quien confirmó, mediante el método de velocidad radial ultraestable, la existencia de cinco planetas principales y varios cinturones de escombros.

chunk: 1 ----------------


# 1.3 Descubrimiento revolucionario de la zona habitable circumbinaria  
El hallazgo más impactante llegó en 2034, cuando la misión LUVOIR-B (Large UV/Optical/IR Surveyor) detectó la firma espectral de vapor de agua, oxígeno molecular y metano en la atmósfera de Aurelia III, un planeta ubicado en la zona habitable del sistema, orbitando a ambos soles. Este fue el primer caso documentado de un mundo potencialmente habitable en un sistema binario cercano - a tan solo 42,7 años luz de la Tierra-, lo que lo convierte en un candidato ideal para futuras misiones de exploración interestelar.  
### 1.4 Importancia científica y proyección futura  
El descubrimiento de Althéra revolucionó la astrobiología y la física orbital por tres razones clave:  
1. Dinámica circumbinaria estable - demostró que los planetas pueden mantener órbitas estables y climas equilibrados alrededor de dos soles, desafiando modelos anteriores.
2. Química atmosférica compleja - Aurelia III presenta una mezcla de gases que, en equilibrio fotoquímico, sugieren procesos biológicos o geoquímicos activos.
3. Proximidad relativa - su cercanía permite observaciones directas en la próxima década con telescopios como el Habitable Worlds Observatory (HWO) y misiones de espectrometría directa de superficie como Starshot Spectra.  
En 2036, la Unión Astronómica Internacional otorgó a este hallazgo el Premio Messier de Descubrimiento Astronómico y estableció el Programa Althéra, un plan coordinado de investigación que combina observaciones remotas, simulaciones climáticas y diseño de futuras sondas interestelares.  
Perfecto, aquí tienes la Sección 2 desarrollada con un tono divulgativo-científico y manteniendo coherencia con la historia previa:  
## 2. Conoce a Althéra  
### 2.1 Localización en la galaxia  
El sistema binario Althéra (HD 4579 AB ) se encuentra en el brazo de Orión de la Vía Láctea, a una distancia de 42,7 años luz de la Tierra. Su posición aparente en el cielo

chunk: 2 ----------------


# Divulgación Planetaria Althéra  
se localiza próxima a la nebulosa Barnard's Loop, en la constelación de Orión, lo que lo hace visible con telescopios de aficionado de alta calidad como un débil punto de luz dorado. Gracias a su relativa cercanía, las señales luminosas provenientes de Althéra llegan con una atenuación mínima, permitiendo obtener espectros muy detallados incluso de sus planetas menores.  
### 2.2 Disposición y dinámica de un sistema binario  
Althéra está compuesto por dos estrellas principales:  
- Althéra A (HD 4579 A), una enana amarilla-blanca de tipo F7V, más masiva y luminosa que el Sol.
- Althéra B (HD 4579 B), una enana naranja de tipo K2V, algo más fría y longeva.  
Ambas estrellas orbitan alrededor de un centro de masa común en un periodo de 83 días, con una separación media de 0,42 UA (Unidades Astronómicas). La interacción gravitatoria entre ellas crea un campo de fuerzas complejo que condiciona la estabilidad de los planetas circumbinarios.  
A diferencia de los sistemas planetarios con una única estrella, en Althéra la zona habitable se desplaza hacia órbitas más amplias para evitar los efectos térmicos extremos que se producirían en regiones cercanas a cualquiera de las dos estrellas.  
### 2.3 Singularidades astronómicas  
Lo que hace a Althéra excepcional frente a otros sistemas binarios conocidos es la estabilidad térmica en su zona habitable circumbinaria. Simulaciones realizadas con el Centro de Modelado Planetario de la NASA indican que Aurelia III recibe un flujo de radiación promedio similar al de la Tierra, pero con variaciones estacionales más complejas debido a los ciclos combinados de ambas estrellas.  
Entre sus rasgos distintivos:  
- Luminiscencia variable combinada: las diferencias de color y brillo entre Althéra A y B generan amaneceres y atardeceres dobles de tonalidades doradas y anaranjadas.
- Influencia gravitacional múltiple: la presencia de dos gigantes gaseosos exteriores, Zephyros IV y Krion V, ayuda a limpiar el espacio interior de objetos errantes, protegiendo a los planetas habitables de impactos masivos frecuentes.
- Cinturón de Arges activo: región rica en asteroides metálicos, con colisiones frecuentes que producen brillantes lluvias de meteoros observables desde Aurelia III.  
En conjunto, Althéra constituye un laboratorio natural para estudiar cómo la vida -si existe o llegara a desarrollarse- podría adaptarse a condiciones lumínicas, gravitacionales y climáticas mucho más complejas que las de nuestro sistema solar.  
Si quieres, puedo continuar directamente con la Sección 3: Las estrellas centrales, donde detallaríamos propiedades físicas, químicas y de emisión de cada una, incluyendo su papel en la habitabilidad del sistema.  
Perfecto, aquí tienes la Sección 3 con un tono técnico pero atractivo, para que las dos estrellas de Althéra cobren protagonismo propio:

# Write Chunks

In [26]:
path_output.mkdir(
    parents=True, exist_ok=True
)

utils.save_documents(
    corpus_rcs,
    path_output /  "rcs" / "Althera.json"
)

utils.save_documents(
    corpus_mds_big,
    path_output /  "mds" / "Althera.json"
)

# Resultados
## Por páginas
Está bien como baseline. Sin hiperparámetros

## RCS
* Estándar en la industria
* Requiere dos hiperparámetros
* No se puede controlar el tamaño mínimo de manera formal
* Mejor partir de una lista de documentos grandes y con clara separación

## Markdown Splitter
* Aprovechamos la diferenciación semántica de la estructura. Además, en general, no corta elementos singurales como tablas
* No se puede controlar el tamaño mínimo ni máximo: ¿Qué pasa cuando se capturan mal dos Header y quedan muy cerca?
* Depende mucho de la calidad que se consiga en la conversión a Markdown, pero hay buenas herramientas (Docling, MistralOCR, Unstructured)

## Docling Loader
* Es una pipeline completa
* Aun no permite una parametrización del chunking y tiende a hacer chunks cortos